# Import Library 

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm import tqdm
from datetime import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Conv2D, Flatten
from tensorflow.keras.applications import MobileNetV2

#### Set Seed 

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

#### Config 

In [3]:
DATA_PATH = f'../Datasets/kfood/'
CKPT_PATH = f'./checkpoints/baseline_learning_model.h5'
LOG_PATH = './logs/baseline_model/' + datetime.now().strftime("%Y%n%d-%H%M%S")

size = 224
batch_size = 64

classes = 150

learning_rate = 1e-2
wd = 0.0005
max_lr = 1e-2
min_lr = 5e-5
cycle_len = 20

EPOCHS = 100
use_pretrained = True

# Load Data

In [4]:
# featurewise_center=True,
# featurewise_std_normalization=True,
# rotation_range=20,
# width_shift_range=0.2,
# height_shift_range=0.2,
# horizontal_flip=True,
# vertical_flip=True,
datagen = ImageDataGenerator(
    rescale=1./255.,
    dtype=np.float32
)

datagenerator = datagen.flow_from_directory(
    DATA_PATH,
    target_size=(size, size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True, 
)

Found 150513 images belonging to 150 classes.


# Load Model 

In [5]:
base_model = MobileNetV2(
        input_shape=(size, size, 3),
        include_top=False,
        weights='imagenet',
    )
base_model.trainable = False

def Baseline(base_model):
    x = inputs = Input([size, size, 3])
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(classes, activation='softmax')(x)
    return Model(inputs=inputs, outputs=output)

with tf.device('/device:GPU:1'):
    model = Baseline(base_model)
    if use_pretrained:
        model.load_weights(CKPT_PATH)

In [6]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)              

# Compile

In [7]:
optimizer = tfa.optimizers.AdamW(learning_rate, wd)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [8]:
ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=CKPT_PATH,
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

def trainfle_fn(x):
    return 1. / (2.**(x - 1))
clr_f = tfa.optimizers.CyclicalLearningRate(
    initial_learning_rate=(max_lr+min_lr)/2,
    maximal_learning_rate=max_lr,
    step_size=cycle_len,
    scale_fn=trainfle_fn
)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(clr_f)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    min_delta=0,
    patience=30,
    verbose=2,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=LOG_PATH,
    histogram_freq=1
)

callbacks = [ckpt, lr_scheduler, early_stop, tensorboard]

# Training 

In [9]:
history = model.fit(
    datagenerator,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/100
   1/2352 [..............................] - ETA: 0s - loss: 3.3791 - accuracy: 0.1562WARNING:tensorflow:From /home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
  13/2352 [..............................] - ETA: 8:58 - loss: 3.5586 - accuracy: 0.1538

/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


  48/2352 [..............................] - ETA: 9:51 - loss: 3.4047 - accuracy: 0.1855

/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:792: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


 285/2352 [==>...........................] - ETA: 8:48 - loss: 3.3161 - accuracy: 0.1880

...

1547/2352 [==================>...........] - ETA: 3:35 - loss: 3.2966 - accuracy: 0.1881

/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41486
  "Possibly corrupt EXIF data.  "
/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41487
  "Possibly corrupt EXIF data.  "


1634/2352 [===================>..........] - ETA: 3:12 - loss: 3.2974 - accuracy: 0.1880

/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 37386
  "Possibly corrupt EXIF data.  "
/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 37396
  "Possibly corrupt EXIF data.  "


2352/2352 [==============================] - ETA: 0s - loss: 3.2996 - accuracy: 0.1867
Epoch 00001: accuracy improved from -inf to 0.18667, saving model to ./checkpoints/baseline_learning_model.h5
2352/2352 [==============================] - 629s 267ms/step - loss: 3.2996 - accuracy: 0.1867
Epoch 2/100
2352/2352 [==============================] - ETA: 0s - loss: 3.3193 - accuracy: 0.1839
Epoch 00002: accuracy did not improve from 0.18667
2352/2352 [==============================] - 603s 256ms/step - loss: 3.3193 - accuracy: 0.1839
Epoch 3/100
2352/2352 [==============================] - ETA: 0s - loss: 3.3229 - accuracy: 0.1800
Epoch 00003: accuracy did not improve from 0.18667
2352/2352 [==============================] - 599s 255ms/step - loss: 3.3229 - accuracy: 0.1800
Epoch 4/100
2352/2352 [==============================] - ETA: 0s - loss: 3.3352 - accuracy: 0.1797
Epoch 00004: accuracy did not improve from 0.18667
2352/2352 [==============================] - 600s 255ms/step - loss: